In [23]:
# Install the transformers library if not already installed
!pip install transformers

# Import necessary libraries
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import tensorflow as tf
import numpy as np
import torch
from torch import nn

# Create a class called GrammarCorrection
class GrammarCorrection:

    def __init__(self):
        # Initialize the model and the tokenizer.
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    # Define a method to detect anomalies in original tokens
    def check(self, dataset, label, best_probability, checklist):
        result = []
        final_sentences = []
        final_labels=[]
        # initialize labels
        corrected=0
        checklist=self.tokenizer.encode(checklist[0], return_tensors="pt", add_special_tokens=True)
        # Initialize the Softmax function
        m = nn.Softmax(dim=1)

        # Loop through each sentence in the dataset
        for index, eval_sentence in enumerate(dataset):

            # Tokenize the input sentence and convert it to PyTorch tensor
            inputs = self.tokenizer(eval_sentence, return_tensors="pt")
            tokens_tensor = self.tokenizer.encode(eval_sentence, add_special_tokens=False, max_length=1020, truncation=True)
            sentence = []  # Initialize a list to store the tokens of the sentence after checking grammar
            sentence.append(tokens_tensor[0]) # append the first word in the sentence
            # Generate model outputs
            outputs = self.model(**inputs, labels=inputs["input_ids"])

            theirprobabilities = []

            # Loop through tokens in the sentence
            for i in range(1, len(tokens_tensor)):
                pre_word = tokens_tensor[i - 1]
                curr_word = tokens_tensor[i]

                # Calculate word probabilities using Softmax
                softmax_output = m(outputs[1][:, i - 1, :])
                word_probability = softmax_output[0][curr_word]
                word_probability = word_probability.detach().numpy()
                # Check if word probability is below a threshold
                if word_probability < best_probability:
                    if curr_word in checklist:
                        probabilities = []

                        # Calculate probabilities for specific words in theirList
                        for j in range(len(checklist.tolist()[0])):
                            p = softmax_output[0][checklist.tolist()[0][j]]
                            p = p.detach().numpy()
                            probabilities.append(p)
                        # Find the index of the highest probability
                        max_prob_index = np.argmax(probabilities)

                        # Check if the highest probability is different from the current word's index
                        if max_prob_index != checklist.tolist()[0].index(curr_word):
                            sentence.append(checklist.tolist()[0][max_prob_index])
                            corrected=1
                        else:
                            sentence.append(curr_word)

                    else:
                        sentence.append(curr_word)

                else:
                    sentence.append(curr_word)

            print(self.tokenizer.decode(sentence))  # Decode the sentence
            final_sentences.append(sentence)
            final_labels.append(corrected)
            sentence = []  # Reset the sentence list
            corrected=0 # Reset the label
        return final_sentences, final_labels

In [19]:
# Import the random module
import random

def read_data_add_perturbation(url):
    # Open the file for reading
    file = open(url, "r")

    # Read the content of the file into a list
    content = file.readlines()

    # Create an empty dataset list to store sentences
    dataset = []

    # Iterate through each sentence in the file
    for sentence in content:
        # Check if 'there' is in the sentence and add it to the dataset
        if 'there' in sentence:
            dataset.append(sentence)
        # Check if 'their' is in the sentence
        if 'their' in sentence:
            # Add the sentence to the dataset
            dataset.append(sentence)

    # Close the file
    file.close()

    # Create a label list with zeros initially
    label = [0] * len(dataset)

    # Randomly select a subset (1/4) of indices to perturb
    result = random.sample(range(0, len(dataset)), round(len(dataset) / 4))

    # Sort the dataset in reverse order (to safely delete elements)
    dataset.sort(reverse=True)

    # Iterate through selected indices for perturbation
    for number in result:
        # Check if 'their' is in the sentence at the selected index
        if 'their' in dataset[number]:
            # Replace 'their' with 'there'
            new_sentence = dataset[number].replace("their", "there")
            # Add the new sentence to the dataset
            dataset.append(new_sentence)
            # Update the label to 1 (perturbed)
            label.append(1)
            # Delete the original sentence from the dataset and label
            del dataset[number]
            del label[number]
        else:
            # Replace 'there' with 'their'
            new_sentence = dataset[number].replace("there", "their")
            # Add the new sentence to the dataset
            dataset.append(new_sentence)
            # Update the label to 1 (perturbed)
            label.append(1)
            # Delete the original sentence from the dataset and label
            del dataset[number]
            del label[number]

    # Return the dataset and corresponding labels
    return dataset, label


In [ ]:
#generate the dataset and labels
dataset, label=read_data_add_pertubation('/content/drive/MyDrive/Vantage_lab/preprocessed_movie_lines.txt')

In [45]:
# Combine X and y into a single list of tuples
data = list(zip(dataset, label))
# Shuffle the data randomly
np.random.shuffle(data)
# Split the shuffled data back into X and y
X_shuffled, y_shuffled = zip(*data)
# Convert them back to lists (if needed)
dataset = list(X_shuffled)
label = list(y_shuffled)

In [61]:
print(len(dataset))

285


In [24]:
theirList=["There there There their Their"]
model = GrammarCorrection()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [54]:
corrected, prediction=model.check(dataset, label, 0.1, theirList)

is there something wrong with the phone?

put your right foot there --

there's nothing there, you brain- dead cunt. think about it. There's absolutely no way in this world to connect us to anything. i want you to hang the phone up, and if you call me about this again i'm going to send a friend of mine out there and have him crack you open with a fucking rib spreader.

it there's one thing i don't need advice on, it's how to drive.

you've asked why i stay a bachelor? There goes the best reason i know!

if they actually bothered to bury her.

tamina was a friend of mine. my shower was broken, she let me use theirs.

can't you see... look out there. nobody's got their porch lights on anymore. it's too late. those hoyt kids are just out to get in trouble. see, they put shaving cream all over that mailbox there. they chopped down herb crane's new bushes. little bastards. jesus h. christ.

shep, no! i 'll bet you have exciting things happen all the time down there.

simon, i didn't want it

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(label, prediction)

0.8456140350877193

In [57]:
from sklearn.metrics import fbeta_score
fbeta_score(label, prediction, average='micro', beta=0.5)

0.8456140350877193

In [58]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(label, prediction)

In [59]:
print(fpr)
print(tpr)

[0.    0.125 1.   ]
[0.         0.73770492 1.        ]


In [60]:
from sklearn.metrics import roc_auc_score
roc_auc_score(label, prediction)

0.8063524590163935